In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, InputLayer, Dense, Flatten, Reshape, LeakyReLU, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from cv2 import imread, imshow, destroyAllWindows, waitKey, resize, imwrite
import numpy as np
import os
from sklearn.model_selection import train_test_split


In [4]:
image_list_X = []
image_list_Y = []


In [6]:
def load_data(data_dir, size):
    count = 0
    for filename in os.listdir(data_dir):
        image = imread(os.path.join(data_dir, filename))
        height, width, c = image.shape
        image = resize(image, dsize=size)
        image_list_X.append(image)
        count += 1
        print(count, " - ", filename)
        if count == 1250:
            print("Durduruldu")
            break
    
    print("Dosyalar Yüklendi")

load_data("F:\\Restoresyon\\data\\mix", (256,256))


1  -  -church-brick-place-of-worship-avenue-english-scottish-estate-modest-1216073.jpg
2  -  10002vetton_ru_steppe-and-forest-4608x3456.jpg
3  -  1015351_wallpaper-clouds-river-forest-landscape-mountain-nature_2560x1600_h.jpg
4  -  102982-pustynya-otrazhenie-ozero-pejzazhi_gor-voda-2560x1440.jpg
5  -  1075624756_w640_h640_kartina-na-holste.jpg
6  -  13082021aa2e0af9f15af9f9e65c0293c181dd7c.jpg
7  -  13625880.png
8  -  15-152740_green-grass.jpg
9  -  150962391459fb086aa45c62.75954367.jpg
10  -  1548112910-201902300-mountain-nevseoboi.com.ua.jpg
11  -  1619708613608acac538e4b7.89110832.jpg
12  -  1674192791_ogorodniku-com-p-peizazhi-prirodi-foto-12.jpg
13  -  1674192825_ogorodniku-com-p-peizazhi-prirodi-foto-24.jpg
14  -  1678093120_gas-kvas-com-p-fon-prirodi-dlya-risunka-krasivii-26.jpg
15  -  1678477656_sneg-top-p-oboi-peizazhi-priroda-les-instagram-53.jpg
16  -  17302771.png
17  -  17836b7698244921e54d21f449512edd.jpg
18  -  190147.jpg
19  -  1920x1280_D093D0BED180D0BDD0BED0B5-D0BED0B

In [16]:
processed_images_X = np.array(image_list_X)
processed_images_Y = np.array(image_list_Y)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (703,) + inhomogeneous part.

In [11]:
high_res_images = processed_images_X.astype('float32') / 255.0
low_res_images = processed_images_Y.astype('float32') / 255.0

NameError: name 'processed_images_X' is not defined

In [7]:
low_res_images = low_res_images.reshape(-1, 256, 256, 3)
high_res_images = high_res_images.reshape(-1, 512, 512, 3)



In [10]:
len(image_list_Y)

703

In [16]:
model = Sequential()

model.add(InputLayer(shape=(None, None, 3)))

model.add(Conv2D(64, (9, 9), activation='relu', padding='same'))
    
#model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2DTranspose(32, (3, 3), padding='same', activation='relu'))  
model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))


model.add(Conv2D(3, (5, 5), activation='tanh', padding='same'))  
    
# model.add(Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', activation='relu'))  
# model.add(Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
    
    
# model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    
# model.add(UpSampling2D(size=(2, 2)))
    
# model.add(Conv2DTranspose(3, (3, 3), strides=(1, 1), padding='same')) 

In [18]:
model.compile(optimizer='adam',loss='mse')

callbacks = [EarlyStopping(monitor="val_loss", patience=50)]

In [19]:
model.fit(x=low_res_images, 
    y=high_res_images,
    callbacks=callbacks,
    batch_size=1,
    epochs=15,
    validation_split=0.15)

Epoch 1/15


ValueError: Dimensions must be equal, but are 512 and 256 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, sequential_2_1/conv2d_8_1/Tanh)' with input shapes: [1,512,512,3], [1,256,256,3].

In [12]:
model.save("SR_test3.h5")

In [ ]:
def predict_model(image): 
    image1 = (np.array(image).astype("float32")/255)
    image1= image1.reshape(-1, image.shape[0], image.shape[1], 3)
    image2 = model.predict(image1)


    image2 = image2.reshape(image2.shape[1], image2.shape[2], image2.shape[3])
    cur = np.zeros((image2.shape[0], image2.shape[1], 3), dtype=np.float32)
    cur[:,:,:] = image2[:, :, :]
    return cur


In [ ]:
image = imread("Test\\test8.jpeg")
print("shapes: ", image.shape[:])

predicted = predict_model(image)
predicted = predict_model(predicted)

print("shapes: ", predicted.shape[:])



In [ ]:
from PIL import Image
 

imshow("org", image)


imshow("predicted", predicted)

img = Image.fromarray(predicted)
img.save("sr.png")

waitKey()
destroyAllWindows()

# trying SuperRes-GAN Model


In [ ]:



# Giriş boyutunu belirle
input_shape = (128, 128, 3)  # Örneğin, (256, 256, 3)

# Ayırt edici modeli oluştur
disc = Sequential()

disc.add(InputLayer(input_shape))
disc.add(Conv2D(64, kernel_size=3, strides=1, padding='same'))
disc.add(LeakyReLU(alpha=0.2))
disc.add(Conv2D(128, kernel_size=3, strides=1, padding='same'))
disc.add(BatchNormalization())
disc.add(LeakyReLU(alpha=0.2))
disc.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
disc.add(BatchNormalization())
disc.add(LeakyReLU(alpha=0.2))
disc.add(Flatten())
disc.add(Dense(1024))
disc.add(LeakyReLU(alpha=0.2))
disc.add(Dense(1, activation="sigmoid"))

# Ayırt edici modelinin özetini görüntüle
disc.summary()


# CREATE GAN MODEL

In [ ]:
gan = Sequential()
gan.add(model)
gan.add(disc)
gan.compile(loss='binary_crossentropy', optimizer="adam")


In [ ]:
# Eğitim parametreleri
num_epochs = 1
batch_size = 1

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}:")
    
    
    
    for batch_i in range(0, len(low_res_images), batch_size):
        # Batch seçimi
        images_lr_batch = low_res_images[batch_i:batch_i+batch_size]
        images_hr_batch = high_res_images[batch_i:batch_i+batch_size]
        
        # Discriminator eğitimi
        d_loss = disc.train_on_batch(images_lr_batch, images_hr_batch)
        
        # Generator eğitimi
        g_loss = gan.train_on_batch(images_lr_batch, images_hr_batch)
        
        # Her bir batch sonunda kayıp değerlerini yazdırma
        print(f"Batch {batch_i//batch_size+1}/{len(low_res_images)//batch_size} -- D Loss: {d_loss}, G Loss: {g_loss}")
